# CHANGE VECTOR ANALYSIS IN POSTERIOR PROBABILITY SPACE

In [1]:
import ee
import json
import os
import geemap

from dotenv import load_dotenv
from utils2 import (create_composite_DW, create_composite_S2, 
                    threshold_optimization, change_type_discrimination,
                    remove_small_objects)

## Initialize Google Earth Engine API

In [2]:
NOTEBOOK_DIR = os.getcwd()

# Get the project root directory (two levels up)
PROJECT_ROOT = os.path.abspath(os.path.join(NOTEBOOK_DIR, '../..'))

# Load environment variables
load_dotenv()

# Get credentials path and ensure it's relative to PROJECT_ROOT
GEE_CREDENTIALS_PATH = os.path.join(PROJECT_ROOT, os.getenv('GEE_CREDENTIALS_PATH'))
GEE_PROJECT_ID = os.getenv('GEE_PROJECT_ID')

# Initialize GEE
credentials = ee.ServiceAccountCredentials(
    '',
    GEE_CREDENTIALS_PATH,
    GEE_PROJECT_ID
)
ee.Initialize(credentials, opt_url="https://earthengine-highvolume.googleapis.com")

## Use GEE Sentinel-2 image with respective Dynamic Land Cover Map

In [3]:
## Change lat / lon by a polygon area
coordinates = [
          [
            [
              -64.76453718063973,
              -32.45727328679724
            ],
            [
              -64.7735675941235,
              -32.495040067045785
            ],
            [
              -64.71535024762193,
              -32.508327610241274
            ],
            [
              -64.7209222048774,
              -32.4642443139475
            ],
            [
              -64.76453718063973,
              -32.45727328679724
            ]
          ]
        ]

In [4]:
# Region of interest defined by coordinates
roi = ee.Geometry.Polygon(coordinates)
T1_date = '2020-01-01'
T2_date = '2021-12-31'

range_days = 10

# Create composites for each date using create_composite_DW function
composite_T1 = create_composite_DW(T1_date, range_days, roi)
composite_T2 = create_composite_DW(T2_date, range_days, roi)

# Compute the change vector of probabilities (for each pixel across probability classes)
delta_prob = composite_T2.subtract(composite_T1)

# Compute the magnitude of the change vector using the Euclidean norm across bands
magnitude = delta_prob.pow(2).reduce(ee.Reducer.sum()).sqrt()

# Convert the image to an array and obtain the index of the maximum along axis 0 (bands)
labels_t1 = composite_T1.toArray().arrayArgmax().arrayGet([0]).rename('labels_t1')
labels_t2 = composite_T2.toArray().arrayArgmax().arrayGet([0]).rename('labels_t2')

# Optimize the threshold
best_threshold, best_Lk = threshold_optimization(magnitude, labels_t1, labels_t2, roi, 10)
print("Best threshold:", best_threshold, "with Lk =", best_Lk)

# --- Generate the change mask ---
# The change mask is defined by applying the optimized threshold to the magnitude image.
changed_mask = magnitude.gte(ee.Image.constant(best_threshold))

# --- Apply change type discrimination ---
# It is assumed that change_type_discrimination works on probability images
# and returns an ee.Image with the transition code (a*100 + b) for each pixel.
change_map = change_type_discrimination(composite_T1, composite_T2, changed_mask, 9)

# --- Example: Extract forest change ---
# It is assumed that the forest class is 1, so transitions with a = 1 will have codes between 100 and 200.
forest_change = change_map.gte(100).And(change_map.lte(200))

# Remove small objects (e.g., objects smaller than 100 pixels)
forest_change_cleaned = remove_small_objects(forest_change, min_size=100, connectivity=8)

# Vectorize the cleaned forest change image and visualize in GeeMap
forest_change_vectors = forest_change_cleaned.updateMask(forest_change_cleaned).reduceToVectors(
    geometry=roi,
    scale=10,
    geometryType='polygon',
    eightConnected=True,
    labelProperty='forest_change',
    reducer=ee.Reducer.countEvery()
)

Dynamic World available images: 3
Dynamic World available images: 4
Best threshold: 0.2920113698752748 with Lk = 22.04322692198884


In [ ]:
# # Retrieve the feature collection as a dictionary
# forest_change_vectors_info = forest_change_vectors.getInfo()

# # Write the dictionary to a GeoJSON file locally
# with open('forest_change_vectors.geojson', 'w') as f:
#     json.dump(forest_change_vectors_info, f)

# # Retreieve the forest change as a numpy array
# forest_change_array = geemap.ee_to_numpy(forest_change, region=roi)

# ## Save the forest change array as a GeoTIFF
# geemap.numpy_to_geotiff(forest_change_array, 'forest_change.tiff', roi.getInfo()['coordinates'], crs='EPSG:4326')

In [5]:
# Create a new geemap interactive map
Map = geemap.Map()

# Center the map on the region of interest (roi) with zoom level 10
Map.centerObject(roi, 10)

# Create a composite Sentinel-2 image for time T1 and clip it to the ROI
S2_T1 = create_composite_S2(T1_date, range_days, roi)
S2_T1_clip = S2_T1.clip(roi)

# Create a composite Sentinel-2 image for time T2 and clip it to the ROI
S2_T2 = create_composite_S2(T2_date, range_days, roi)
S2_T2_clip = S2_T2.clip(roi)

# Add the T1 composite image to the map with a natural-color visualization (RGB using bands B4, B3, B2)
Map.addLayer(
    S2_T1_clip, 
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 
    'S2 T1'
)

# Add the T2 composite image to the map with the same visualization parameters
Map.addLayer(
    S2_T2_clip, 
    {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 
    'S2 T2'
)

# Add the forest change vectors layer, painting the vectors on an empty image.
# The stroke width is set to 2 pixels and the color is red ('FF0000').
Map.addLayer(
    ee.Image().paint(forest_change_vectors, 0, 2),
    {'palette': 'FF0000'},
    'Forest Change Vectors'
)

# Add the region of interest (ROI) as a blue outline for reference.
Map.addLayer(
    ee.Image().paint(roi, 0, 2), 
    {'palette': 'blue'}, 
    'ROI'
)

# Display the map
Map

Sentinel-2 available images: 3
Sentinel-2 available images: 4


Map(center=[-32.48227999382408, -64.74328619065847], controls=(WidgetControl(options=['position', 'transparent…

### Bibliography

```
@ARTICLE{5597922,
  author={Chen, Jin and Chen, Xuehong and Cui, Xihong and Chen, Jun},
  journal={IEEE Geoscience and Remote Sensing Letters}, 
  title={Change Vector Analysis in Posterior Probability Space: A New Method for Land Cover Change Detection}, 
  year={2011},
  volume={8},
  number={2},
  pages={317-321},
  keywords={Remote sensing;Pixel;Radiometry;Accuracy;Training;Satellites;Earth;Change vector analysis (CVA);land cover change;postclassification comparison (PCC);posterior probability space},
  doi={10.1109/LGRS.2010.2068537}}

```